# Chapter 3: Undirected graphs

*Selected Topics in Mathematical Optimization*

*2016-2017*

**Bernard De Baets**

**Michiel Stock**

**Raúl Pérez-Fernández**

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy.spatial import voronoi_plot_2d, Voronoi
import numpy as np
%matplotlib inline

## Some essential data structures in python

Using the correct datastructure is key to implementing an efficient graph algorithm. Below is a concise overview of some elementary python data structures.

For more information on the time complexity of operations on these data structures, see [this](https://wiki.python.org/moin/TimeComplexity) link.

### Lists: ordered  collection of objects

Lists store objects in sequential order. Adding and removing elements at the back is easy, inserting or removing elements elsewhere is computationally inefficient (memory has to be reshuffled).

In [ ]:
my_list = [1, 2, 3, 4, 5]

In [ ]:
my_list.append(6)  # add element at the back
my_list

In [ ]:
my_list.pop()  # return last element

In [ ]:
my_list  # element is gone

In [ ]:
my_list[4]  # take element

In [ ]:
my_list[3] = 'three'  # replace
my_list

In [ ]:
# for loops in lists
squares = [x**2 for x in range(1, 11)]
squares

In [ ]:
# a more complex example
squares_odd = [x**2 for x in range(1, 11) if x%2==1]
squares_odd

### Sets: unordened collection of objects

Sets have no ordening and cannot contain duplicates. It is very efficient (constant time compelxity) to check if an element is present and to add or remove an object.

In [ ]:
numbers = set([1, 2, 15, -1, 4, 2, 3, 4])  # make set from list 
numbers  # no particular order or duplicates

In [ ]:
5 in numbers  # check if element is member

In [ ]:
numbers.add(5)  # add element

In [ ]:
5 in numbers

In [ ]:
# set operations on two sets
new_numbers = set([2, 15, -1, 8, 6, 15])
new_numbers

In [ ]:
numbers & new_numbers  # intersection (AND)

In [ ]:
numbers | new_numbers  # union (OR)

In [ ]:
numbers - new_numbers  # set difference (in first but not in second)

## Dictorinary: key-value storage

Dictionaries are the best of both lists and sets, they store values by a key

In [ ]:
my_dict = {'A' : 1, 'B' : 'bee', 'C' : [1, 2, 3]}  # make dictionary

In [ ]:
my_dict['B']  # get value associated by key

In [ ]:
my_dict['D'] = 'new value'  # add new key-value
my_dict

In [ ]:
'A' in my_dict  # check if dict has key

In [ ]:
for k, v in my_dict.items():  # iterate over values
    print(k, v)

In [ ]:
for k in my_dict.keys():  # iterate over keys
    print(k)

In [ ]:
for v in my_dict.values():  # iterate over values
    print(v)

## Representing graphs in python

Consider the following example graph:

Typically, graphs are represented as:
- **list of edges**: a set of edges (tuples of vertices) we use sets are there is no order in the edges. THis is the prefered format to *store* graphs.
- **adjacency lists**: in python implemented as dictionaries. Each vertex is a key and the associated value is a set of neighboring edges (for undirected graphs every outgoing edge should be balanced by an ingoing edge).
- **adjacenty matrices**: matrix of which the elements store the edges. Is only used for dense graphs.

Example of the theory:

![](Figures/undir_graph.png)

In [ ]:
edges = set([('A', 'B'), ('B', 'C'), ('C', 'D'), ('A', 'D'), ('C', 'E'), ('D', 'E')])

In [ ]:
from graphs import edges_to_adj_list

In [ ]:
adj_list = edges_to_adj_list(edges)
adj_list

Some basic graph algorithms are implemented:
- `give_connected_component`: gives the largest connected component for a vertex
- `is_connected`: checks if a graph is connected (every edge can be reached from every other edge)
- `has_path`: checks if there exists a path between two vertices
- `is_bridge`: checks if an edge is a bridge (i.e. removing it makes the graph disconnected

Run the examples and check the result

In [ ]:
from graphs import give_connected_component, is_connected, is_bridge, has_path

In [ ]:
example1_edges = set([('A', 'B'), ('A', 'C'), ('B', 'C'), ('D', 'C'), ('D', 'E'),
                     ('E', 'F'), ('F', 'G'), ('G', 'E'), ('D', 'F')
                     ])
example2_edges = set([('A', 'B'), ('A', 'C'), ('B', 'C'), ('D', 'E'),
                     ('D', 'G'), ('F', 'G'), ('G', 'E'), ('D', 'F')
                     ])

In [ ]:
example1_adj_list = edges_to_adj_list(example1_edges)
example2_adj_list = edges_to_adj_list(example2_edges)

In [ ]:
give_connected_component(example1_adj_list, 'A')

In [ ]:
give_connected_component(example2_adj_list, 'A')

In [ ]:
is_connected(example1_adj_list)

In [ ]:
is_connected(example2_adj_list)

In [ ]:
is_bridge(example1_adj_list, ('A', 'B'))

In [ ]:
is_bridge(example1_adj_list, ('C', 'D'))

In [ ]:
example1_adj_list

In [ ]:
has_path(example2_adj_list, 'A', 'C')

In [ ]:
has_path(example2_adj_list, 'A', 'F')

## Fleury's algorithm

Fleury's algorithm is a simple (though inefficient algorithm to find an Eulerian trail (i.e. a walk on the graph that passes all the edges).

It can be shown that a graph has an Eulerian trail if and only if it is connected and it it has exactly zero or two edges with an odd degree.

Fleury's algorithm is executed as follows:
1. Start at a vertex of odd degree, or, if the graph has none, start with an arbitrarily chosen vertex.
2. Chooses the next edge in the path to be one whose deletion would not disconnect the graph (i.e. is not a bridge), unless there is no such edge, in which case pick the remaining edge left at the current vertex.
3. Add the edge to the path and remove the edge from the graph.
4. Repeat steps 2-3 until no more edges are left.

**ASSIGNMENT 1**

1. Complete the pseudocode of fleury's algorithm. The algorithm first checks if an Eulerian path exists (and returns None if not) and returns the path as a list. Use the given function `is_bridge` to check if an 
2. Study the time complexity of the algorithm. For this you have 25 graphs at your disposal. These graphs are stored in json format in the form `eulerianxxx.json` with '`xxx`' the number of edges. The function `load_graph` can read this data file (see example). For each file, measure the time to find the Eulerian path and make a graph (in log-log scale) of the time vs. the number of edges. Comment on the relation (linear time complexity, super/sublinear?) Does this make sense? HINT: detecting if an edge is a bridge is done with an average time complexity linear in the number of edges.

In [ ]:
def fleury_algorithm(adj_list):
    """
    Fleury's algorithm for finding an Eulerian path (if one exists)
    
    First it checks if the path exists, if it exists, it returns a list of
    the nodes that are visisted in an Eulerian path
    
    Inputs:
        - adj_list
        
    Outputs:
        - path
    """
    adj_list = {k : v.copy() for k, v in adj_list.items()}  # make a copy
    # check if Eulerian path exists
    
    # if not, return None
    
    # else, compute the path and return it

In [ ]:
fleury_algorithm(adj_list)

In [ ]:
from graphs import load_graph

In [ ]:
# example on how to load the graph
adj_loaded = load_graph('Data_Eulerian/eulerian14.json')
adj_loaded

In [ ]:
# use this cell to measure the time to find the Eulerian path for each graph

In [ ]:
# use this cell to plot the time as a function of the number of edges (log-log scale)

DISCUSS THE EMPIRICAL TIME COMPLEXITY HERE

## Graph coloring using six colors

Graph coloring is an optimization problem of assinging a color to each label using as few colors as possible such that no neighboring vertices have the same color.

The four color theorem states that a planar graph can be colored using only four colors.

We will use six colors (though we rarely need more than five) to color a planar graph. For this, the algorithm is very simple:

1. Color recursively with the six colors all vertices of degree five or more.
2. Color the rest of the graph with a free color.

**ASSIGNMENT 2**

1. Complete the graph coloring algorithm.
2. What do you think the time complexity of this algorithm is going to be (based on reasoning).

You can illustrate this algorithm on a Voronoi plot (a tesselation of a plane based on the distance to a set of points). Feel free to improve the graphics :-)

In [ ]:
def graph_color(adj_list, colors=['k', 'r', 'b', 'g', 'y', 'm']):
    """
    Provides one of six colors for each vertex in a planar graph
    such that no neighboring vertices have the same color
    
    Inputs:
        - adj_list: adjecency list of a planar graph
        - colors: list of six colors (default those for matplotlib plotting)
        
    Output:
        - vertices_colors: a dict with a color for each vertex
    """
    vertices_colors = {}
    
    # complete the algorithm
    
    return vertices_colors

In [ ]:
from graphs import get_planar_graph

In [ ]:
X = np.random.randn(20, 2)  # random points for Voronoi tesselation
planar_graph = get_planar_graph(X)

In [ ]:
planar_graph

In [ ]:
coloring = graph_color(planar_graph, colors=['k', 'r', 'b', 'g', 'y', 'm'])
coloring

In [ ]:
fig, ax = plt.subplots()

vor = Voronoi(X)

voronoi_plot_2d(vor, ax=ax, show_points=False, show_vertices=False)
for i, c in coloring.items():
    ax.scatter(X[i, 0], X[i, 1], c=c, s=50)